In [289]:
import numpy as np
from get_data import xor_data
from get_data import donut_data
from get_data import fig_data
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import time

Xtrain,Ytrain,Xtest,Ytest=fig_data(limit=10000,ratio=0.5)
#idx = np.logical_or(Ytrain == 4., Ytrain == 5.)
#Xtrain = Xtrain[idx]
#Ytrain = Ytrain[idx]
#idx = np.logical_or(Ytest == 4., Ytest == 5.)
#Xtest = Xtest[idx]
#Ytest = Ytest[idx]
X=Xtrain
Y=Ytrain

In [292]:
count_x=sorted(list(set(Xtrain[:,300])))
#print(np.shape(Xtrain))
#count_x

In [293]:
def gini(Y):
    list_y=[]
    list_j=[]
    sumj=0.0
    count_y=sorted(list(set(Y)))
    #print(count_y,Y)
    for j in count_y:
        
        jj=np.array([j]*len(Y))
        cj=((np.sum(Y==jj))/len(Y))**2.0
        #print(cj,len(Y),np.sum(Y[Y==j]))
        list_y.append(cj)
        list_j.append(j)
        sumj=cj+sumj
        
    zipped=zip(list_j, list_y)    
    #print(list_y)
    #print(list_j)
    return 1.0-1.0*sumj,sorted(zipped, key=lambda x: x[1])[-1][0]  
#mixing=[1.0,2.0,3.0,4.0,5.0]
#gini(Ytrain)


In [294]:
def information_Gain(i,j,Yright,Yleft,current_entropy):
    
    nleft=len(Yleft)
    nright=len(Yright)
    N=nleft+nright
    #_,ent_r=entropy(Yright)
    #_,ent_l=entropy(Yleft)
    
    ent_r,_=gini(Yright)
    ent_l,_=gini(Yleft)
    IG=current_entropy-(nright/N)*ent_r-(nleft/N)*ent_l
    return IG

In [295]:
#X,Y   split
start=time.time()
#X=Xtrain[0:100,:]
#Y=Ytrain[0:100]

    
def best_split(X,Y):
    
    D=np.shape(X)[1]
    N=np.shape(X)[0]

    current_entropy,pred_temp=gini(Y)
    IG_max=-1
    best_col=None
    best_cut=None
    Ypred=-1

    for i in range(D):          #loop through features

        count_x=sorted(list(set(X[:,i])))    #choose whether or not want the first and last item in since become whole if you cut
                                            # then for majority zero it will be empty count_x_new=count_x[1:-1]
        for j in count_x:

            Xleft_=X[X[:,i]<=j,i]   #can do j+(j+1)/2 for  mid points
            Yleft_=Y[X[:,i]<=j]

            Xright_=X[X[:,i]>j,i]
            Yright_=Y[X[:,i]>j]    
            

            if len(Yright_) == 0 or len(Yleft_) == 0:   #make sure is dividing

                continue

            IG=information_Gain(i,j,Yright_,Yleft_,current_entropy)

            if IG>IG_max:

                best_col=i
                best_cut=j
                IG_max=IG
                Ypredict=pred_temp
            else:

                pass
            

    return best_col,best_cut,current_entropy,IG_max
    
#best_col,best_cut,current_entropy,IG_max=best_split(X,Y)    

end=time.time()
#print(end-start)
#print(best_col,best_cut,current_entropy,IG_max)

In [296]:
best_col,best_cut,current_entropy,IG_max=best_split(X,Y)
np.shape(X)

(5000, 784)

In [297]:
def split_data(best_col,best_cut,X,Y):
    
    Xleft=X[X[:,best_col]<=best_cut,:]   #can do j+(j+1)/2 for  mid points
    Yleft=Y[X[:,best_col]<=best_cut]            

    Xright=X[X[:,best_col]>best_cut,:]   #can do j+(j+1)/2 for  mid points
    Yright=Y[X[:,best_col]>best_cut]  
    return Xright,Xleft,Yright,Yleft

In [298]:
Xright,Xleft,Yright,Yleft=split_data(best_col,best_cut,X,Y)
print(np.shape(Xright),np.shape(Xleft))

(2245, 784) (2755, 784)


In [299]:
def count_pred(Y):
    
    classes,counts=np.unique(Y,return_counts=True)
    return classes[np.argmax(counts)]

In [300]:
count_pred(Y)

1.0

In [301]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, Y):
        self.predictions = count_pred(Y)
        #self.col = None
        #self.split = None
        #self.left = None
        #self.right = None
        

In [302]:
class decision_node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,best_col,best_cut,left_branch,right_branch):
        
        self.best_col=best_col
        self.best_cut=best_cut
        self.left_branch=left_branch
        self.right_branch=right_branch       

In [303]:
def build_tree(X,Y):
    
    
    if ((len(Y)==1) or len(set(Y))==1):
        
        return Leaf(Y)  # return prediction argmax Y
        #best_col=None   ?
        #best_cut=None
        #left_branch=None
        #right_branch=None
        
        #-> leaf node ->  predict Y[0]  node.right=None, node.left=None
      
    else:    
        #current_entropy,pred_temp=gini(Y)
        best_col,best_cut,current_entropy,IG_max=best_split(X,Y)    #-> need y predict
        
    if IG_max==0.0:
        
        return Leaf(Y)
        #best_col=None     ?
        #best_cut=None
        #left_branch=None
        #right_branch=None
            

        #else:
    #print(best_col,best_cut,IG_max)
    Xright,Xleft,Yright,Yleft=split_data(best_col,best_cut,X,Y)
    #print(Yleft)
    left_branch=build_tree(Xleft,Yleft)
    right_branch=build_tree(Xright,Yright)
           
    return decision_node(best_col,best_cut,left_branch,right_branch)

In [304]:
my_tree = build_tree(X,Y)

In [284]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing, node.best_col,node.best_cut)

    # Call this function recursively on the true branch
    print (spacing + '--> Left:')
    print_tree(node.left_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> Right:')
    print_tree(node.right_branch, spacing + "  ") 

In [309]:
print_tree(my_tree)

 569 0.0
--> Left:
   430 0.0
  --> Left:
     378 0.6
    --> Left:
       154 0.0
      --> Left:
         485 0.08235294117647059
        --> Left:
           376 0.10980392156862745
          --> Left:
             157 0.0
            --> Left:
               499 0.0
              --> Left:
                 596 0.0
                --> Left:
                   433 0.796078431372549
                  --> Left:
                     130 0.0
                    --> Left:
                       187 0.996078431372549
                      --> Left:
                         488 0.9921568627450981
                        --> Left:
                           349 0.32941176470588235
                          --> Left:
                             180 0.996078431372549
                            --> Left:
                               186 0.8745098039215686
                              --> Left:
                                Predict 7.0
                              --> Right:
           

                  Predict 6.0
                --> Right:
                  Predict 9.0
            --> Right:
               540 0.0
              --> Left:
                 156 0.25098039215686274
                --> Left:
                   95 0.0
                  --> Left:
                    Predict 8.0
                  --> Right:
                    Predict 2.0
                --> Right:
                  Predict 1.0
              --> Right:
                 517 0.33725490196078434
                --> Left:
                   70 0.0
                  --> Left:
                    Predict 8.0
                  --> Right:
                    Predict 1.0
                --> Right:
                  Predict 2.0
          --> Right:
             213 0.06274509803921569
            --> Left:
               207 0.0
              --> Left:
                 98 0.0
                --> Left:
                  Predict 8.0
                --> Right:
                  Predict 1.0
            

In [305]:
def predict_sample(node, X):
    """See the 'rules of recursion' above."""
    #print(node)
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if X[node.best_col] <= node.best_cut:
        return predict_sample(node.left_branch,X)
    else:
        return predict_sample(node.right_branch,X)

In [306]:
Y_pred=[]
for i,xx in enumerate(Xtrain):
    
    #print ("Actual: %s. Predicted: %s" %
    yy=predict_sample(my_tree, xx)
    Y_pred.append(yy)
    #print(Ytrain[i], yy)
print('score train',np.mean(Ytrain==Y_pred))    

#score train 1.0   0-1
#score train 1.0   2-6
#score train 1.0   9-1
#score train 1.0   8-9
#score train 1.0   4-5
#score train 1.0   0 to 9

score train 1.0


In [307]:
Y_pred=[]
for i,xx in enumerate(Xtest):
    
    #print ("Actual: %s. Predicted: %s" %
    yy=predict_sample(my_tree, xx)
    Y_pred.append(yy)
    #print(Ytest[i], yy)
print('score test',np.mean(Ytest==Y_pred)) 
#score test 0.9895833333333334  0-1
#score test 0.883248730964467   2-6
#score test 0.9473684210526315   9-1
#score test 0.881578947368421   8-9
#score test 0.875               4-5
#score test 0.663               0 to 9   1000 sample
#score test 0.769               0 to 9   5000 sample

score test 0.769
